In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# Define quantile loss function
def quantile_loss(y_true, y_pred, quantile=0.6):
    e = y_true - y_pred
    return tf.keras.backend.mean(tf.keras.backend.maximum(quantile * e, (quantile - 1) * e), axis=-1)

# Load data
df = pd.read_csv('IESCO_dataset.csv')
df.dropna(inplace=True)

# Preprocess data
s1 = MinMaxScaler(feature_range=(0,1))
Xs = s1.fit_transform(df)
s2 = MinMaxScaler(feature_range=(0,1))
ys = s2.fit_transform(df[['target_variable']])

window= 5
X=[]
Y=[]
for i in range(window,len(Xs)):
    X.append(Xs[i-window:i,:])
    Y.append(ys[i])
X,Y = np.array(X), np.array(Y)

In [2]:
# Split data into training and testing sets
X_train1, X_test1, y_train, y_test = train_test_split(X, Y, test_size=0.3, shuffle=False)
print('Train set shape', X_train1.shape)
print('testing set shape', X_test1.shape)

Train set shape (9186, 5, 5)
testing set shape (3937, 5, 5)


In [3]:
def hybrid_model(input_shape, quantile=0.6):
    inputs = tf.keras.Input(shape=input_shape)
    
    # First LSTM layer
    lstm_out = tf.keras.layers.LSTM(32, return_sequences=True)(inputs)
    
    # Second LSTM layer
    #lstm_out = tf.keras.layers.LSTM(64, return_sequences=True)(lstm_out)

    # First Attention layer
    attention = tf.keras.layers.Attention()([lstm_out, lstm_out])
    attention1 = tf.keras.layers.GlobalAveragePooling1D()(attention)

    dropout = tf.keras.layers.Dropout(0.25)(attention1)



    # Output layer
    quantile_lstm_out = tf.keras.layers.Dense(1)(attention1)
    
    # Quantile LSTM model
    quantile_lstm_model = tf.keras.Model(inputs=inputs, outputs=quantile_lstm_out)
    quantile_lstm_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                                loss=lambda y_true, y_pred: quantile_loss(y_true, y_pred, quantile=quantile))
    
    return quantile_lstm_model

In [4]:
model = hybrid_model((X_train1.shape[1], X_train1.shape[2]))
history = model.fit(X_train1, y_train, epochs=50, batch_size=32)

Epoch 1/50
288/288 [==============================] - 5s 7ms/step - loss: 0.0294
Epoch 2/50
288/288 [==============================] - 2s 5ms/step - loss: 0.0197
Epoch 3/50
288/288 [==============================] - 2s 5ms/step - loss: 0.0160
Epoch 4/50
288/288 [==============================] - 2s 5ms/step - loss: 0.0149
Epoch 5/50
288/288 [==============================] - 2s 6ms/step - loss: 0.0144
Epoch 6/50
288/288 [==============================] - 2s 7ms/step - loss: 0.0139
Epoch 7/50
288/288 [==============================] - 2s 7ms/step - loss: 0.0144
Epoch 8/50
288/288 [==============================] - 2s 7ms/step - loss: 0.0136
Epoch 9/50
288/288 [==============================] - 2s 6ms/step - loss: 0.0134
Epoch 10/50
288/288 [==============================] - 2s 6ms/step - loss: 0.0134
Epoch 11/50
288/288 [==============================] - 2s 6ms/step - loss: 0.0136
Epoch 12/50
288/288 [==============================] - 3s 11ms/step - loss: 0.0133
Epoch 13/50
288/288 [===

In [5]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 5, 5)]       0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 5, 32)        4864        ['input_1[0][0]']                
                                                                                                  
 attention (Attention)          (None, 5, 32)        0           ['lstm[0][0]',                   
                                                                  'lstm[0][0]']                   
                                                                                                  
 global_average_pooling1d (Glob  (None, 32)          0           ['attention[0][0]']          

In [6]:
y_pred = model.predict(X_test1)

124/124 [==============================] - 2s 3ms/step


In [7]:
# Reshape y_pred and y_test from 3D to 2D
y_pred_reshaped = y_pred.reshape(-1, y_pred.shape[-1])
#y_test_reshaped = y_test.reshape(-1, y_test.shape[-1])

In [8]:
y_pred = s2.inverse_transform(y_pred_reshaped)
y_test = s2.inverse_transform(y_test)

In [9]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)*100
mse = mean_squared_error(y_test, y_pred)
nrmse = (rmse/np.mean(y_pred))*100

print("RMSE:", rmse)
print("MAE:", mae)
print("MAPE:", mape)
print("MSE:", mse)
print("NRMSE:", nrmse)

RMSE: 64.62628977165718
MAE: 41.60963710855644
MAPE: 4.0640950501920186
MSE: 4176.557329650202
NRMSE: 6.224101238021502


In [10]:
y_pred= pd.DataFrame(y_pred)
y_test= pd.DataFrame(y_test)

In [12]:
result_df = pd.concat([y_pred, y_test], axis=1)
result_df.to_csv(r"E:\MODELS\IESCO\DAQ-LSTM_1.csv", index=False)